In [1]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importiere Daten

project_person_relations = pd.read_csv("project_person_relations.csv", encoding = "utf-8") # für Anzahl Projekt / Person
people_joined_with_institutions = pd.read_csv("people_joined_with_institutions.csv", encoding = "utf-8") # für top 10 researchers per Uni
project_gender_index = pd.read_csv('data/project_gender_index.csv', encoding = 'utf-8') # für Berechnung des Gender Index je Person / Institution

FileNotFoundError: [Errno 2] File b'people_joined_with_institutions.csv' does not exist: b'people_joined_with_institutions.csv'

In [3]:
# Berechne person_gender_index basierend auf dem Durchschnitt des project_gender_indexes der Projekte, an der die Person teilnimmt

## Füge project_gender_index über project_id zu project_person_data hinzu
project_gender_index['project_id'] = project_gender_index.project_id.astype(int)
project_gender_index_map = project_gender_index.set_index(['project_id']).to_dict()['gender_index']  
project_person_relations['gender_index'] = project_person_relations.set_index('project_id_number').index.map(project_gender_index_map)   

## Gruppiere project_person_data nach Person & Projekten und entferne alle Projekte, für die es keinen project_gender_index gibt
person_gender_index = pd.DataFrame(project_person_relations.groupby(['person_id', 'project_id_number'])['gender_index'].first())
person_gender_index = person_gender_index.dropna() # drop projects that have no gender_index for calculation (basically, we're ignoring the same projects we've ignored before)

## Berechne Durchschnitt von project_gender_index pro Person
person_gender_index = person_gender_index.mean(level=['person_id'])

## Speichere person_gender_index
person_gender_index = person_gender_index.reset_index()
person_gender_index.to_csv('data/person_gender_index.csv')

In [4]:
# Berechne institution_gender_index basierend auf dem Durchschnitt des person_gender_index der Personen, die der Institution angehörig sind

## Entferne alle Zeilen in people_joined_with_institutions, bei denen die institution_id fehlt
print(people_joined_with_institutions['institution_id'].isna().sum())
print(people_joined_with_institutions['institution_id'].notna().sum())
reduced_people_joined_with_institutions = people_joined_with_institutions.dropna(subset=['institution_id'])

## Füge person_gender_index über person_id zu people_joined with_institutions hinzu
institution_gender_index = reduced_people_joined_with_institutions
person_gender_index_map = person_gender_index.set_index(['person_id']).to_dict()['gender_index']  
institution_gender_index['person_gender_index'] = institution_gender_index.set_index('person_id').index.map(person_gender_index_map)

# Gruppiere people_joined_with_institutions nach Institution & Person und entferne alle Personen, für die es keinen person_gender_index gibt
print(institution_gender_index['person_gender_index'].isna().sum())
print(institution_gender_index['person_gender_index'].notna().sum())
institution_gender_index = pd.DataFrame(institution_gender_index.groupby(['institution_id', 'person_id'])['person_gender_index'].first())
institution_gender_index = institution_gender_index.dropna(subset=['person_gender_index'])

## Berechne Durchschnitt von person_gender_index pro Institution
institution_gender_index = institution_gender_index.mean(level=['institution_id'])

## Speichere person_gender_index
institution_gender_index = person_gender_index.reset_index()
institution_gender_index.to_csv('data/institution_gender_index.csv')

14687
44178
21238
22940


In [5]:
# Erstelle TOP 10 researchers Liste pro Institution

## Gruppiere project_person_data nach Person & Projekten, zähle Anzahl der Projekte 
project_count_per_person = pd.DataFrame(project_person_relations.groupby(['person_id'])['project_id_number'].count())
project_count_per_person = project_count_per_person.rename(columns={'project_id_number': 'project_count'})
project_count_per_person = project_count_per_person.reset_index()

## Füge project_count über person_id zu reduced_people_joined_with_institutions hinzu
institution_people_project_count = reduced_people_joined_with_institutions
project_count_map = project_count_per_person.set_index(['person_id']).to_dict()['project_count']  
institution_people_project_count['project_count'] = institution_people_project_count.set_index('person_id').index.map(project_count_map)

## Berechne Top 10 Researchers nach Anzahl der Projekte
top_researchers_by_projects = pd.DataFrame(institution_people_project_count.groupby(['institution_id', 'person_id'])['project_count'].first())
top_researchers_by_projects = top_researchers_by_projects.sort_values([('project_count')], ascending=False)
top_researchers_by_projects = top_researchers_by_projects.groupby(level=0).apply(lambda df: df.sort_index(by='project_count')[-10:])

## Speichere Top 10 researchers
top_researchers_by_projects.to_csv('data/top_researchers_by_projects.csv')

## Nice to have: Berechne Top 10 Researchers nach Person Gender Index
#top_researchers_by_gender_index = pd.DataFrame(institution_people_project_count.groupby(['institution_id', 'person_id'])['person_gender_index'].first())
#top_researchers_by_gender_index = top_researchers_by_gender_index.dropna(subset=['person_gender_index'])
## In der folgenden Zeile müssten wir dann die 10 Werte pro Institution bekommen, die am nächsten um 0,5 liegen
#top_researchers_by_gender_index = top_researchers_by_gender_index.groupby(level=0).apply(lambda df: df.sort_index(by='person_gender_index')[-10:])